In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

In [69]:
IMAGE_SIZE=128
BATCH_SIZE=8
EPOCHS=20
CHANNELS=3

In [45]:
from collections import Counter
import os

train_dir = 'dataset_tomato_classified_oversampled/oversampled_train'
class_names = ['Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']
class_counts = {}

for class_name in class_names:
    class_path = os.path.join(train_dir, class_name)
    class_counts[class_name] = len(os.listdir(class_path))

print("Class distribution:", class_counts)
print("Total samples:", sum(class_counts.values()))

Class distribution: {'Tomato_Bacterial_spot': 2246, 'Tomato_Early_blight': 2246, 'Tomato_Late_blight': 2246, 'Tomato_Leaf_Mold': 2246, 'Tomato_Septoria_leaf_spot': 2246, 'Tomato_Spider_mites_Two_spotted_spider': 2246, 'Tomato__Target_Spot': 2246, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 2246, 'Tomato__Tomato_mosaic_virus': 2246, 'Tomato_healthy': 2246}
Total samples: 22460


In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
        'dataset_tomato_classified_oversampled/oversampled_train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode="sparse",
#         save_to_dir="C:\\Code\\potato-disease-classification\\training\\AugmentedImages"
)

Found 22459 images belonging to 10 classes.


In [51]:
train_generator.class_indices


{'Tomato_Bacterial_spot': 0,
 'Tomato_Early_blight': 1,
 'Tomato_Late_blight': 2,
 'Tomato_Leaf_Mold': 3,
 'Tomato_Septoria_leaf_spot': 4,
 'Tomato_Spider_mites_Two_spotted_spider': 5,
 'Tomato__Target_Spot': 6,
 'Tomato__Tomato_YellowLeaf__Curl_Virus': 7,
 'Tomato__Tomato_mosaic_virus': 8,
 'Tomato_healthy': 9}

In [53]:
class_names = list(train_generator.class_indices.keys())
class_names

['Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [55]:
count=0
for image_batch, label_batch in train_generator:
#     print(label_batch)
    print(image_batch[0])
    break
#     count+=1
#     if count>2:
#         break

[[[0.52118593 0.47412714 0.47412714]
  [0.5210205  0.47396162 0.47396162]
  [0.52044487 0.4733861  0.4733861 ]
  ...
  [0.47191826 0.44838884 0.45623198]
  [0.29853845 0.27500904 0.28285217]
  [0.4265489  0.4030195  0.41086262]]

 [[0.46687847 0.41981962 0.41981962]
  [0.474936   0.4278772  0.4278772 ]
  [0.4829935  0.4359347  0.4359347 ]
  ...
  [0.44601908 0.42248967 0.4303328 ]
  [0.31753114 0.29400173 0.30184487]
  [0.42416182 0.40063238 0.40847555]]

 [[0.54394627 0.49688745 0.49688745]
  [0.53243554 0.4853767  0.4853767 ]
  [0.5209248  0.47386596 0.47386596]
  ...
  [0.42011994 0.39659053 0.40443367]
  [0.3365239  0.31299445 0.3208376 ]
  [0.42070857 0.39717916 0.4050223 ]]

 ...

 [[0.68175656 0.6621487  0.650384  ]
  [0.6693194  0.64971155 0.63794684]
  [0.6832397  0.66363186 0.65186715]
  ...
  [0.55321044 0.5453673  0.557132  ]
  [0.562419   0.55457586 0.56634057]
  [0.5716276  0.5637845  0.5755492 ]]

 [[0.68808746 0.6684796  0.6567149 ]
  [0.66413957 0.6445317  0.632767  ]


In [59]:
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)
validation_generator = validation_datagen.flow_from_directory(
        'dataset_tomato_classified_oversampled/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode="sparse"
)

Found 1597 images belonging to 10 classes.


In [61]:
test_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        'dataset_tomato_classified_oversampled//test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode="sparse"
)

Found 3211 images belonging to 10 classes.


In [75]:
# Define input shape (adjust IMAGE_SIZE as needed)
IMAGE_SIZE = 128  # Increase if needed based on your data
CHANNELS = 3      # Assuming RGB images
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 10

model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Flatten(),  # Removed extra pooling layers
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 16, 16, 64)      

In [77]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [89]:
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

In [91]:
steps_per_epoch

2808

In [93]:
validation_steps

200

In [85]:
# Calculate steps per epoch
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = validation_generator.samples // BATCH_SIZE


In [99]:
history = model.fit(
    train_generator,
    steps_per_epoch=2808,
    batch_size=BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=200,
    verbose=1,
    epochs=20,
)

Epoch 1/20
2808/2808 [==============================] - 823s 289ms/step - loss: 0.9039 - accuracy: 0.6826 - val_loss: 0.4104 - val_accuracy: 0.8585
Epoch 2/20
2808/2808 [==============================] - 382s 136ms/step - loss: 0.3812 - accuracy: 0.8674 - val_loss: 0.3247 - val_accuracy: 0.8898
Epoch 3/20
2808/2808 [==============================] - 218s 77ms/step - loss: 0.2716 - accuracy: 0.9065 - val_loss: 0.2552 - val_accuracy: 0.9117
Epoch 4/20
2808/2808 [==============================] - 197s 70ms/step - loss: 0.1995 - accuracy: 0.9309 - val_loss: 0.2164 - val_accuracy: 0.9292
Epoch 5/20
2808/2808 [==============================] - 209s 74ms/step - loss: 0.1680 - accuracy: 0.9441 - val_loss: 0.3542 - val_accuracy: 0.8923
Epoch 6/20
2808/2808 [==============================] - 198s 71ms/step - loss: 0.1467 - accuracy: 0.9489 - val_loss: 0.2144 - val_accuracy: 0.9330
Epoch 7/20
2808/2808 [==============================] - 186s 66ms/step - loss: 0.1339 - accuracy: 0.9559 - val_loss:

In [100]:
scores = model.evaluate(test_generator)

402/402 [==============================] - 68s 170ms/step - loss: 0.3613 - accuracy: 0.9377


In [101]:
# Save the model in HDF5 format
model.save('tomato_oversampled.h5')